In [ ]:
!pip install -q tiktoken openai langchain sentence_transformers fuzzywuzzy spacy langchain-openai langchain_experimental
!pip install --upgrade --quiet langchain-openai langchainhub

In [ ]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
os.environ["OPENAI_API_KEY"]=""

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain.memory import ConversationBufferWindowMemory
from openai import OpenAI
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import spacy
import os

In [ ]:
# 테스트 실행
test_cases = [
    {
        "query": "사고유형 '보1'의 과실비율은 어떻게 되나요?",
        "correct_answer": "A70 : B30"
    },
    {
        "query": "신호기가 없는 횡단보도에서 횡단보도를 통과하려는 차량이 횡단보도를 건너고 있던 보행자를 충격한 사고의 과실비율은?",
        "correct_answer": "A0 : B100"
    },
    {
        "query": "보행자가 녹색신호에 횡단을 시작했지만 적색신호에 충격을 받은 경우의 과실비율은?",
        "correct_answer": "A10 : B90"
    },
    {
        "query": "사고유형 '보12'의 사고상황을 설명해주세요.",
        "correct_answer": "신호기가 있는 횡단보도를 통과하여 교차로에 진입하려는 차량이 보행자신호등 녹색점멸신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 녹색점멸 신호가 여전히 켜져 있는 상황에서 충격한 사고이다."
    },
    {
        "query": "보행자신호가 적색일 때 횡단을 시작한 보행자와 녹색신호에 직진하는 차량이 충돌한 경우의 과실비율은?",
        "correct_answer": "A70 : B30"
    },
    {
        "query": "사고유형 '거42'의 사고상황이 뭐야?",
        "correct_answer": "자전거횡단도를 횡단하는 A자전거와 직진하여 자전거횡단도를 통과하는 B차량이 충돌한사고"
    },
    {
        "query": "신호기에 의해 교통정리가 이루어지고 있는 교차로에서 적색신호에 직진하는 A 차량과 교차로에서 녹색화살표 신호에 좌회전하는 B차량이 충돌한 사고의 과실비율은?",
        "correct_answer": "100:0"
    },
    {
        "query": "애니메이션 https://accident.knia.or.kr/video/car15-1.mp4 의 사고상황이 뭐야?",
        "correct_answer": "신호기에 의해 교통정리가 이루어지고 있지 않는 교차로에서 직진하는 A차량과 맞은편 주 행방향에서 좌회전하는 B차량이 충돌한 사고"
    },
    {
        "query": "앞지르기(추월)가 금지된 도로에서 선행하는 A차량과 동일방향에서 후행하다가 A차량을 추월하면서 A차량의 전방으로 진입(앞지르기) 하려는 B차량이 충돌한 사고의 관련법규가 뭐야?",
        "correct_answer": "도로교통법 제20조(진로양보의무) / 도로교통법 제21조(앞지르기 방법 등) / 도로교통법 제60조(갓길 통행금지 등) / 도로교통법 제22조 (앞지르기 금지 장소)"
    },
    {
        "query": "일반 도로의 가장자리(갓길 포함) 또는 도로와 보도의 가장자리에 동시에 걸치고 주·정차 중인 A차량을 동일방향에서 후행하여 진행하는 B차량이 추돌한 사고의 애니메이션은?",
        "correct_answer": "https://accident.knia.or.kr/video/car42-2.mp4"
    },
    {
        "query": "회전 2차로형 이상 회전 교차로 안으로 양 차량이 동시에 진입하면서, 1차로(좌측)에서 진입하는 A차량과 2차로(우측)에서 진입하는 B차량이 충돌한 사고에 대한 판결사례는?",
        "correct_answer": "등록되지 않은 참고판례입니다."
    },
    {
        "query": "2차로형 교차로에서 내부 1차로를 주행하다 2차로로 진출하려는 A차량과 회전교차로로 진입하려는 B차량과의 사고의 판결사례는?",
        "correct_answer": "서울중앙지방법원 2017.12.7.선고 2017나47290 판결. 과실비율은 40:60"
    },
    {
        "query": "사고유형 '거3-4'의 판결사례는?",
        "correct_answer": "서울중앙지방법원 2015.9.10. 선고 2015가단5001249 판결. B차량의 과실 60%."
    },
    {
        "query": "사고유형 '보35'의 과실해설은?",
        "correct_answer": "대법원 2007.7.13. 선고 2007다 26240 판결. 자동차 운전자 과실이 없다. / 서울고등법원 2007.1.18. 선고 2006나40739 판결. B차량 과실20% / 서울지방법원 1999.10.29. 선고 99가단84270 판결. B차량 과실 0%"
    },
    {
        "query": "과실비율이 '70:30'인 사고유형은?",
        "correct_answer": "보1,보8,차8-2,차13-3,차17-2,차8-2,차13-3,차17-2,거10-2,차1-5,차14-1,차20-2,차47-2,차14-1"
    },
    {
        "query": "판결사례가 '등록되지 않은 참고판례입니다.'인 것 중 사고유형이 '차3-8'인 애니메이션은?",
        "correct_answer": "https://accident.knia.or.kr/video/car3-8.mp4"
    },
    {
        "query": "교차로에서 녹색신호에 따라 직진하는 A차량과 상시유턴구역에서 유턴하는 B차량이 충돌한 사고 관련 도로교통법규는 뭘까",
        "correct_answer": "도로교통법 제5조(신호 또는 지시에 따를 의무) / 도로교통법 제18조(횡단 등의 금지) / 도로교통법 제38조(차의 신호)"
    },
    {
        "query": "차량이 직진 중 육교 및 지하도 부근에서 도로를 건너고 있던 보행자를 충격한 사고에 대한 과실해설 해줘",
        "correct_answer": "육교나 지하도가 설치된 경우에는 횡단보도가 설치된 경우보다 운전자가 보행자의 존재를 예견하기 어렵다는 점을 감안하여 보행자의 기본과실을 40%"
    },
    {
        "query": "신호기에 의한 교통정리가 한쪽 방향에만 이루어지고 있는 교차로에서 신호기가 있는 도로를 이용하여 직진하는 B차량과 신호기가 없는 도로를 이용하여 직진(좌회전)하는 A차량 이 충돌한 사고에 대한 과실 비율은?.",
        "correct_answer": "내용 없음"
    },
    {
        "query": " 2차로형 회전교차로에서 회전하다가 우측으로 진출하기 위해 진로변경 중인 A차량과 교차로 내에서 회전 중인 B차량이 충돌한 사고의 과실해설 알려주세요.",
        "correct_answer": "회전교차로에서는 진출을 위하여 언제든지 진로변경을 할 가능성이 높다는 점이 예견가능하고, 또한 같이 회전하고 있는 차량끼리 양보 의무가 가중된다고 할 수 있으므로, 일반적인 진로변경 사고보다 진로변경차량의 과실비율을 약간 낮춰서 양 차량의 기본 과실비율을 60:40으로 정한다."
    },
    {
        "query": "신호기가 있는 횡단보도를 통과하여 교차로에 진입하려는 차량이 보행자신호등 녹색점멸신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 녹색점멸 신호가 여전히 켜져 있는 상황에서 충격한 사고의 사고유형은?",
        "correct_answer": "https://accident.knia.or.kr/video/car3-8.mp4"
    },
    {
        "query": "애니메이션이 https://accident.knia.or.kr/video/car3-5.mp4 인 사고유형은?",
        "correct_answer": "차3-5"
    },
    {
        "query": "애니메이션이 https://accident.knia.or.kr/video/car9-2.png 인 사고내용은?",
        "correct_answer": "신호기에 의해 교통정리가 이루어지고 있지 않고 한쪽에 일시정지 표지가 있는 교차로에 서 일시정지 표지를 위반하여 직진하는 A차량과 일시정지 표지가 없는 교차로에서 우회전 하는 B차량이 충돌한 사고"
    },
    {
        "query": "애니메이션이 https://accident.knia.or.kr/video/car32-1.mp4 의 판결사례는?",
        "correct_answer": "대법원 1992. 7. 28. 선고 92도1137 판결. 자동차의 운전자에게 과실이 있다고 할 수 없다.(교행중 차량의 주의의무) / 서울중앙지방법원 2017. 3. 31. 선고 2016나66409 판결. B과실 50% / 서울중앙지방법원 2017. 7. 9. 선고 2017나58951 판결. B과실 70% / 부산지방법원 2018. 5. 17. 선고 2017나62471 판결. 중앙선침범 B차량 70%, 진출 A차량 30% 인정(주차장 램프구간 교행)"
    },
    {
        "query": "과실해설이 '도로교통법 제26조 제4항에 따라 직진차량이 좌회전차량보다 우선권이 있지만, 신호기 없는 교차로에서의 사고이므로 도표 324를 준용하되 자전거는 통상 저속으로운행하므로 B차량으로서는 이를 발견하여 사고의 발생을 회피할 수 있다는 점 및 자전거는 차량에 비하여 상대방에게 가해의 위험성이 현저히 낮다는 점을 감안하여 양측의 기본과실을 10:90으로 정한다.'인 사고유형은?",
        "correct_answer": "거8-1"
    },
    {
        "query": "사고유형이 '차9-2'인 사고의 과실비율은?",
        "correct_answer": "A80 : B20"
    },
    {
        "query": "사고내용이 신호기에 의해 교통정리가 이루어지고 있지 않는 동일 폭의 교차로에서 좌회전하는 A차량과 A차량의 진행방향 왼쪽 도로에서 좌회전 진입하는 B차량이 충돌한 사고이다. 인 사고의 과실비율은?",
        "correct_answer": "A40 : B60"
    },
    {
        "query": "사고내용이 신호기에 의해 교통정리가 이루어지고 있는 교차로에서 서로 다른 도로를 이용하여 적색신호에 교차로에 진입하여 직진하는 A자전거와 적색신호에 진입하여 직진하는 B차량이충돌한 사고의 사고유형은?",
        "correct_answer": "거1-5"
    },
    {
        "query": "사고유형이 '차53-1'의 애니메이션과 과실비율은?",
        "correct_answer": "https://accident.knia.or.kr/video/car53-1.mp4, A100:B0 "
    },
    {
        "query": "사고내용이 오른쪽 3차로에서 2차로로 진로변경을 하는 A차량과 왼쪽 1차로에서 2차로로 진로변경을 하는 B차량이 충돌한 사고의 과실해설과 과실비율은?",
        "correct_answer": "양 차량이 좌우에서 동시에 진로변경을 하던 중에 발생한 사고로서 양 차량 모두 진로변경방법 위반의 과실이 있는 것이므로 양 차량의 기본 과실비율을 동등하게 보아 50:50으로 정한다, A50 : B50"
    },
]


In [ ]:
prompt = """

** 당신은 도로교통법 분야에서 20년 이상의 경력을 가진 숙련된 변호사입니다. 다양한 교통사고 사례를 다루어 온 경험과 깊은 전문 지식을 바탕으로,
    교통사고 관련 질문에 대해 전문적이고 실용적인 법률 조언을 제공하는 역할을 맡고 있습니다. 상황을 분석하여 과실비율, 적용 가능한 법률, 관련 판례, 실제 사례를 참조하여 상세한 답변을 제공해 주세요.

** 모든 대화는 한국어로 진행됩니다.

** 답변 시 다음 사항을 반드시 포함해 주세요:

    ** I. 사용자와의 초기 상호작용:
        1. 사용자에게 인사하고, 교통사고 발생 시 '최초 대응 절차'를 안내해 주세요.
            ** 최초 대응 절차 ** (첫 번째 답변 시에만 상세히 제공, 이후 요청 시 간단히 설명):
            - 1. 즉시 정차하세요!
                - 사고 발생 즉시 안전하게 차량을 정차하고, 사고 상황을 확인하세요.
            - 2. 부상자 구호가 우선입니다!
                - 부상자가 있는지 확인하고, 필요시 응급조치 후 119에 신고하세요.
            - 3. 정황 증거를 확보하세요!
                - 사고 현장을 여러 각도로 사진 촬영하고, 목격자를 확보하세요.
            - 4. 경찰서와 보험사에 신고하세요!
                - 가벼운 사고라도 보험사와 경찰에 신고하여 사고를 기록해 두세요.

    ** II. 사용자에게 교통사고 상황에 대해 물어보기:
        1. 상황을 친절하게 물어보고, 사용자 상태를 걱정하는 모습을 보이세요.
        2. 사고의 주요 쟁점을 파악하기 위해 최대 두 번의 추가 질문을 하세요. 필요시 사용자가 제공한 정보로 답변을 구성하세요.

    ** III. 답변 구성:
        유저가 물어 본 내용이 검색한 내용과 95% 이상 일치하면 그 문단에 있는 것 전부 제시하세요
        유저가 물어 본 내용의 유사도가 95% 미만일시 가장 높은 유사도의 질문내용과 같은 문단에 있는 것들을 하단 내용에 맞게 전부 제시하세요
        1. 과실 비율:
            - 청구차량 (과실 비율) : 피청구차량 (과실비율), A (과실비율) : B (과실비율), 나 (과실비율) : 상대방 (과실비율) 형식으로 명확하게 제시하세요.
        2. 사고 상황의 주요 쟁점 분석:
            - 주요 쟁점에 대해 명확히 설명하고, 필요시 추가 정보를 요청하세요.
        3. 결정 이유와 근거:
            - 유사한 사례를 바탕으로 과실 비율 및 결론의 근거를 제시하세요.
        4. 관련 법률 조항 및 해석:
            - 관련 교통법, 판례 등을 제시하며, 설명이 필요할 경우 쉬운 예시를 들어 설명하세요.
        5. 유사한 판례나 사례 분석:
            - 관련 유사 판례나 사례를 1~3개 제시하고, 없을 경우에는 없다고 명확히 설명하세요.
        6. 구체적인 해결 방안 또는 법적 절차 안내:
            - 법적 절차 및 해결 방안을 구체적으로 안내하세요.
        7. 주의해야 할 법적 리스크나 고려 사항:
            - 법적 리스크와 주의사항에 대해 설명하세요.
        8. 결론 요약:
            - 위에서 설명한 내용을 요약하여 사용자에게 이해하기 쉽게 설명하세요.

    ** IV. 최종 결론:
        1. 위에서 답변한 내용을 이해하기 쉽게 요약하여 결론을 제시하세요.
        2. 답변의 출처를 명시해 주세요.
        3. 법적 책임의 한계를 명확히 하고, 추가적인 전문가 상담을 권장하는 문구를 포함하세요.
        4. "이 답변은 일반적인 법률 정보 제공을 목적으로 하며, 구체적인 법률 자문을 대체할 수 없습니다. 귀하의 특정 상황에 대해서는 반드시 자격을 갖춘 변호사와 상담하시기 바랍니다."라는 문구를 포함해 주세요.

** 참고 사항:
- 전문 용어는 쉬운 설명을 덧붙여 주세요.
- 복잡한 법률 개념은 일상적인 예시를 통해 설명해 주세요.
- 객관적이고 중립적인 답변을 제공하고, 개인적인 의견이나 편견은 배제해 주세요.

"""

------------------------------------------------------------------------------

In [ ]:
!pip install -U langchain-openai

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# 모델 평가
def evaluate_model(test_cases, llm):
    # LLM 초기화
    llm = ChatOpenAI(
        # model='ft:gpt-4o-mini-2024-07-18:personal::9ytHMa20',
        model='ft:gpt-4o-mini-2024-07-18:personal::A17ZXztl,
        temperature=0
    )

    results = []

    for item in test_cases:
        query = item['query']
        correct_answer = item['correct_answer']

        # 모델에 쿼리를 입력하여 답변을 생성
        response = llm.invoke(query)

        # 모델의 응답 텍스트 추출
        model_response = response.content if hasattr(response, 'content') else response['text']

        # TF-IDF 벡터화 및 코사인 유사도 계산
        vectorizer = TfidfVectorizer().fit_transform([model_response, correct_answer])
        vectors = vectorizer.toarray()
        cosine_sim = cosine_similarity(vectors)[0, 1]

        if correct_answer in model_response or cosine_sim >= 0.8:
            accuracy = 1
        else:
            accuracy = 0

        # 결과 저장
        result_dict = {
            'query': query,
            'model_response': model_response,
            'correct_answer': correct_answer,
            'cosine_similarity': cosine_sim,
            'accuracy': accuracy
        }

        results.append(result_dict)

    # 결과를 DataFrame으로 변환하여 CSV로 저장
    df_results = pd.DataFrame(results)
    df_results.to_csv('model_evaluation_results_tuning2.csv', index=False, encoding='utf-8-sig')

    return df_results

# 평가 함수 호출
if __name__ == "__main__":

    # model_name = 'ft:gpt-4o-mini-2024-07-18:personal::9ytHMa20'
    model_name = 'ft:gpt-4o-mini-2024-07-18:personal::A17ZXztl'
    results = evaluate_model(test_cases, model_name)

    # 평가 결과 출력
    print("Evaluation Results:")
    print(results)

Evaluation Results:
                                                query  \
0                           사고유형 '보1'의 과실비율은 어떻게 되나요?   
1   신호기가 없는 횡단보도에서 횡단보도를 통과하려는 차량이 횡단보도를 건너고 있던 보행...   
2        보행자가 녹색신호에 횡단을 시작했지만 적색신호에 충격을 받은 경우의 과실비율은?   
3                           사고유형 '보12'의 사고상황을 설명해주세요.   
4   보행자신호가 적색일 때 횡단을 시작한 보행자와 녹색신호에 직진하는 차량이 충돌한 경...   
5                               사고유형 '거42'의 사고상황이 뭐야?   
6   신호기에 의해 교통정리가 이루어지고 있는 교차로에서 적색신호에 직진하는 A 차량과 ...   
7   애니메이션 https://accident.knia.or.kr/video/car15-...   
8   앞지르기(추월)가 금지된 도로에서 선행하는 A차량과 동일방향에서 후행하다가 A차량을...   
9   일반 도로의 가장자리(갓길 포함) 또는 도로와 보도의 가장자리에 동시에 걸치고 주·...   
10  회전 2차로형 이상 회전 교차로 안으로 양 차량이 동시에 진입하면서, 1차로(좌측)...   
11  2차로형 교차로에서 내부 1차로를 주행하다 2차로로 진출하려는 A차량과 회전교차로로...   
12                                사고유형 '거3-4'의 판결사례는?   
13                                 사고유형 '보35'의 과실해설은?   
14                              과실비율이 '70:30'인 사고유형은?   
15  판결사례가 '등록되지 않은 참고판례입니다.'인 것 중 사고유형이 '차3-8'인 애니...   
16  교차로에서 녹